In [ ]:
import pandas as pd
import plotly.express as px
import panel as pn
import datetime as dt

# Load the dataset
data = pd.read_csv("E_dataset.csv")

# Data Preprocessing
data['Order_Date'] = pd.to_datetime(data['Order_Date'])
data.dropna(inplace=True)  # Remove rows with missing values

# Aggregate product quantities for each day
daily_quantity = data.groupby('Order_Date')['Quantity'].sum().reset_index()

# Dashboard Components
kde_plot = pn.pane.Plotly(width=500)
category_plot = pn.pane.Plotly(width=500)
scatter_plot = pn.pane.Plotly(width=500)
discounted_items = pn.pane.Plotly(width=500)
age_plot = pn.pane.Plotly(width=500)
device_plot = pn.pane.Plotly(width=500) 

# Create a slider for selecting months
month_slider = pn.widgets.IntSlider( start=1, end=12, value=1)

# Create a radio button for selecting view type
view_radio = pn.widgets.RadioButtonGroup(options=['monthly', 'yearly'], name='Select View', value='monthly')

# Update the plots function
def update_plots(event):
    selected_view = view_radio.value
    
    if selected_view == 'monthly':
        month = month_slider.value  # Get the selected month from the slider
        # Filter data by month
        monthly_data = data[data['Order_Date'].dt.month == month]
        monthly_data1 = daily_quantity[daily_quantity['Order_Date'].dt.month == month]
        category_sales = monthly_data.groupby('Product_Category')['Sales'].sum().reset_index()
        profitable_products = monthly_data.groupby('Product')['Profit'].sum().nlargest(10)
        discounted_top_products = monthly_data.groupby('Product').apply(lambda x: (x['Discount'].sum() / x['Sales'].sum()) * 100)
    else:
        # Filter data by year
        monthly_data = data
        monthly_data1 = daily_quantity
        category_sales = data.groupby('Product_Category')['Sales'].sum().reset_index()
        profitable_products = data.groupby('Product')['Profit'].sum().nlargest(10)
        discounted_top_products = data.groupby('Product').apply(lambda x: (x['Discount'].sum() / x['Sales'].sum()) * 100)
    
    # KDE-like Plot: Total Quantity of Orders Over Time
    hist_fig = px.line(monthly_data1, x='Order_Date', y='Quantity',
                       title='Total Quantity of Orders Over Date')
    kde_plot.object = hist_fig

    # Pie Plot: Sales with Respect to Product Category
    category_fig = px.pie(category_sales, values='Sales', names='Product_Category', title='Sales by Product Category')
    category_plot.object = category_fig

    # Scatter Plot: Most Profitable Products
    scatter_fig = px.scatter(data_frame=profitable_products.reset_index(), x='Product', y='Profit',
                             title='Most Profitable Products', labels={'Profit': 'Total Profit'})
    scatter_plot.object = scatter_fig

    # Bar Plot: Top 10 Products with Highest Discount Percentage
    discounted_top_products = discounted_top_products.nlargest(10)
    discounted_fig = px.bar(data_frame=discounted_top_products.reset_index(), x='Product', y=0,
                            title='Top 10 Products with Highest Discount Percentage',
                            labels={'0': 'Discount Percentage'})
    discounted_items.object = discounted_fig

    # Age Distribution Plot
    age_box_fig = px.box(monthly_data, y='Aging', title='Age Distribution of Customers')
    age_plot.object = age_box_fig

    # Device Type Distribution Plot
    device_fig = px.pie(monthly_data, names='Device_Type', title='Device Type Distribution of Customers')
    device_plot.object = device_fig

# Watch the slider's value changes and call the update_plots function
month_slider.param.watch(update_plots, 'value')
view_radio.param.watch(update_plots, 'value')

# Dashboard Layout
template = pn.template.FastListTemplate(
    title='Customer Segmentation and Product Analysis',
    sidebar=[
        pn.pane.Markdown("# Customer Segmentation and Product Analysis"),
        pn.pane.Markdown("Welcome to Customer Segmentation and Product Analysis Dashboard. The goal is to visually explore and analyze sales, quantities, and discounts of products over time and across different categories. The dashboard provides interactive filters, including a slider to select months and a radio button to choose between monthly and yearly views."),
        pn.Spacer(height=30),
        pn.pane.Markdown("## Filters"),
        pn.pane.Markdown("#### Select a Month"), 
        month_slider,
        pn.Spacer(height=10),
        pn.pane.Markdown("#### Select a View"), 
        view_radio
    ],
    main=[
        pn.Row(
            pn.Column(kde_plot, margin=(0, 25)),
            pn.Column(category_plot, margin=(0, 25))
        ),
        pn.Row(
            pn.Column(scatter_plot, margin=(0, 25)),
            pn.Column(discounted_items, margin=(0, 25))
        ),
        pn.Row(
            pn.Column(age_plot, margin=(0, 25)),
            pn.Column(device_plot, margin=(0, 25))
        ),
    ],

    accent_base_color="#88d8b0",
    header_background="#357EC7",
)

update_plots({"value": 1})  # Initialize plots with the default month
template.servable()

